In [1]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/claim-assignment/claim_assignment.xlsx


In [2]:
pip install openpyxl   

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.2/242.2 KB 1.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
df = pd.read_excel('/kaggle/input/claim-assignment/claim_assignment.xlsx')
df.head()

,MemberID,ClaimNo,LineNo,Bill_Type,Paid_Status,From_date,Thru_date,Revenue_cod,Allowed,Paid
0,1,1,1,111,P,2014-01-04,2014-01-05,120,20000.0,19000.0
1,1,1,2,111,P,2014-01-04,2014-01-05,250,1000.0,1000.0
2,1,2,1,211,P,2014-01-05,2014-01-15,120,4000.0,4000.0
3,2,3,1,111,D,2014-02-01,2014-02-05,120,NaN,NaN
4,2,4,1,111,P,2014-02-01,2014-02-04,111,10000.0,10000.0


In [4]:
df['From_date'] = pd.to_datetime(df['From_date'], errors = 'coerce')
df['Thru_date'] = pd.to_datetime(df['Thru_date'], errors = 'coerce')
df = df[df['Paid_Status'] != 'D']
df = df[df['Bill_Type'] == 111]
df['diff'] = df['Thru_date'] - df['From_date']
df['duration'] = (df['Thru_date'] - df['From_date'])/np.timedelta64(1, 'M')
df = df.reset_index()

In [5]:
def concat(df):
    df_new = pd.DataFrame(columns = ['MemberID', 'From_date', 'Thru_date', 'diff', 'Paid', 'duration'])
    
    i=0
    while i<df.shape[0]:
        j=i
        final_paid=0
        while (df.loc[i].at["MemberID"]== df.loc[j].at["MemberID"] and df.loc[i].at["From_date"]<=df.loc[j].at["From_date"] and df.loc[i].at["Thru_date"]>=df.loc[j].at["Thru_date"]):
            final_paid+=df.loc[j].at['Paid']

            j+=1
            if j==df.shape[0]:
                break
        
        df_new=df_new.append({'MemberID': df.loc[i].at["MemberID"],'From_date': df.loc[i].at["From_date"], 'Thru_date': df.loc[i].at["Thru_date"], 'diff': df.loc[i].at["diff"], 'Paid': final_paid, 'duration': df.loc[i].at["duration"]},ignore_index = True)
        i=i+j
    return(df_new)

df_new= concat(df)

In [6]:
df_new['Final'] = df['Paid']/df['duration']
for i in range (df_new.shape[0]):
    print('For patient',i+1, 'the average inpatient paid dollars for the duration', df_new.loc[i].at['From_date'], 'to',df_new.loc[i].at['Thru_date'],'is $', df_new.loc[i].at['Final'])
    print('')

For patient 1 the average inpatient paid dollars for the duration 2014-01-04 00:00:00 to 2014-01-05 00:00:00 is $ 578300.625

For patient 2 the average inpatient paid dollars for the duration 2014-02-01 00:00:00 to 2014-02-04 00:00:00 is $ 30436.875

